# Projeto 2 - Classificando Automático de Sentimento 

## Por: Beatriz Alexandre e Vitória Almeida

#### Marca escolhida: Lindt

Fomos contratadas pela empresa Lindt, pára analisar os tweets feitos por seus clientessobre sua empresa. Para isso criaremos um programa que irá analisae as mensagens disponiveis como "relevante" ou "irrelevante". Os twers que forem negativos irão disparar um foco na área de marketing.

In [1]:
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib as mpl  
import seaborn as sns
import numpy as np
import itertools
from scipy import stats
from emoji import UNICODE_EMOJI

#### Colocando os dados em um DataFrame

Lendo o arquivo excel

In [2]:
tw = pd.read_excel('tweets_lindt.xlsx', sheet_name = 0)

In [3]:
tw.head()

,Treinamento,Relevante,Positivo/Negativo
0,lindt chocolate is so overrated,1.0,0
1,@elgliko hear that pitter patter in the back g...,0.0,NaN
2,rt @statutory_boy: the guy was addicted to dru...,0.0,NaN
3,"@andreyasasylum @yourmomsuckstho hey, can you ...",1.0,1
4,great job by @noticgroup gms. come and see the...,1.0,1


#### Limpando os tweets

LIMPAR TWEETS

In [4]:
import string
ponto = string.punctuation

#lista para depoi tirar os pontos
po = ["\n", "\t"]

data_limpo = []

#dar espaço nos emojis e limpar os pontos
for tweet in tw["Treinamento"]:
    t = ''
    for palavra in tweet:
        if palavra in UNICODE_EMOJI:
            t = t+ " " + palavra + " "
        elif palavra in po:
            t += " "
        elif palavra not in ponto:
            t += palavra
    data_limpo.append(t)

tw_li =  pd.DataFrame()
tw_li['Treinamento'] = data_limpo
tw_li['Relevante'] = tw['Relevante']

tw_li.head()



,Treinamento,Relevante
0,lindt chocolate is so overrated,1.0
1,elgliko hear that pitter patter in the back gr...,0.0
2,rt statutoryboy the guy was addicted to drugs ...,0.0
3,andreyasasylum yourmomsuckstho hey can you ple...,1.0
4,great job by noticgroup gms come and see them ...,1.0


In [5]:
# tirar os @ e os #
data_limpo = []
x= ' '
for tweet in tw_li['Treinamento']:
    limp = []
    spli = tweet.split()
    for palavra in spli:
        if palavra in UNICODE_EMOJI:
            limp.append(palavra)
        elif len(palavra) > 2 and palavra[0]!='@' and palavra[0] != '#' and palavra[:4] != "http":
            limp.append(palavra)
        elif palavra == tweet[-1] and ("..." in palavra):
            limp = ''
        
    data_limpo.append(x.join(limp))
    
tw_limpo =  pd.DataFrame()
tw_limpo['Treinamento'] = data_limpo
tw_limpo['Relevante'] = tw['Relevante']
tw_limpo['Treinamento'] = tw_limpo['Treinamento'].str.replace('gorgeous\xa0i', 'banana')
#tw_limpo['Treinamento'] = tw_limpo['Treinamento'].str.replace(" ' ' ", " ")

tw_limpo

,Treinamento,Relevante
0,lindt chocolate overrated,1.0
1,elgliko hear that pitter patter the back groun...,0.0
2,statutoryboy the guy was addicted drugs his li...,0.0
3,andreyasasylum yourmomsuckstho hey can you ple...,1.0
4,great job noticgroup gms come and see them and...,1.0
5,blacktwittasa lmao guys screaming 🤣 🤣 🤣 🤣 🤣 🤣 ...,0.0
6,fuck want candy chocolate going ✈ spend all mo...,1.0
7,lindtzwane thank you,1.0
8,lindtzwane misinterpret love sex money 💵 relia...,0.0
9,reebok emailed thank for collab posts with the...,1.0


Dar espaço nos emojis:

Lista de todas as palavras dos tweets:

In [6]:
pala = []

for i in tw_limpo['Treinamento']:
    frase = i.split()
    for p in frase:
        if p not in pala:
            pala.append(p)


Contar quantas vezes aparace o 1(Relevante) e o 0(não relevante):

In [7]:
#contador
Rel = 0
Irre = 0 

for l in range(len(tw_limpo)):
    linha = tw_limpo["Treinamento"][l].split(" ")
    for i in linha:

        if tw_limpo["Relevante"][l] == 1:
            Rel += 1
        else:
            Irre += 1

Dicionários para calcular a frequencia de relevantes e irrelevantes:

In [8]:
#adicionou as palavras no dicionário de frequencia
r_freq = {}
ir_freq = {}

for palavras in pala:
    r_freq[palavras] = 1
    ir_freq[palavras] = 1

for g in range(len(tw_limpo)):
    linha = tw_limpo["Treinamento"][g].split(" ")
    for v in linha:
        if tw_limpo["Relevante"][g] == 0:
            ir_freq[v]+=1
        else:
            r_freq[v]+= 1


A probabilidade de ser relevante da cada palavra

In [9]:
r_prob = {}
ir_prob = {}

#pala é a lista de palavras dos tweets

for palavra in pala:
    r_prob[palavra] = (r_freq[palavra]+1)/(len(pala)+Rel)
    ir_prob[palavra] = (ir_freq[palavra]+1)/(len(pala)+Irre)
    
print(r_prob['lindt'],r_freq[palavra]+1,len(pala),Rel)
print(ir_prob['lindt'],ir_freq[palavra]+1,len(pala),Irre)

0.030652280529671408 2 1906 2172
0.0050335570469798654 3 1906 2266


P(relevante) e P(irrelevante)

In [10]:
irrelevante = 0
relevante = 0 
for m in tw_limpo['Relevante']:
    if m == 0:
        irrelevante += 1
    else:
        relevante += 1 
Pr = relevante/len(tw_limpo['Relevante'])
Pi = 1-Pr




Teoria de Naive - Bayes

# TESTE

In [11]:
g = pd.read_excel('tweets_lindt.xlsx', sheet_name = 1)

In [12]:
g.head()

,Teste,Relevante,Positivo/Negativo
0,rt @thembarumbu: dear ladies\nif you catch ur ...,0,NaN
1,rt @dailytelegraph: one of the most notorious ...,1,0/1
2,commercial law's bitch ...is what i am right n...,0,NaN
3,@hloni_maniers i'm craving lindt lindor 😫,1,1
4,rt @lutho__m: if you're reading this tweet htt...,0,NaN


In [13]:
g.Teste = g.Teste.str.replace('@', '')
g.Teste = g.Teste.str.replace(':', '')
g.Teste = g.Teste.str.replace('-', '')
g.Teste = g.Teste.str.replace('_', '')
g.Teste = g.Teste.str.replace(',', '')
g.Teste = g.Teste.str.replace('!', '')
g.Teste = g.Teste.str.replace('(', '')
g.Teste = g.Teste.str.replace(')', '')
g.Teste = g.Teste.str.replace('=', '')
g.Teste = g.Teste.str.replace('*', '')
g.Teste = g.Teste.str.replace('#', '')

g.head()

,Teste,Relevante,Positivo/Negativo
0,rt thembarumbu dear ladies\nif you catch ur bo...,0,NaN
1,rt dailytelegraph one of the most notorious ba...,1,0/1
2,commercial law's bitch ...is what i am right n...,0,NaN
3,hlonimaniers i'm craving lindt lindor 😫,1,1
4,rt luthom if you're reading this tweet https//...,0,NaN


In [14]:
g_espaco_emoji = []
data = g["Teste"]
for tweet in data:
    t = ''
    for palavra in tweet:
        if palavra in UNICODE_EMOJI:
            t = t+ " " + palavra + " "
        else:
            t += palavra
    g_espaco_emoji.append(t)

g_limpo =  pd.DataFrame()
g_limpo['Teste'] = g_espaco_emoji
g_limpo['Relevante'] = g['Relevante']

g_limpo.head()

,Teste,Relevante
0,rt thembarumbu dear ladies\nif you catch ur bo...,0
1,rt dailytelegraph one of the most notorious ba...,1
2,commercial law's bitch ...is what i am right n...,0
3,hlonimaniers i'm craving lindt lindor 😫,1
4,rt luthom if you're reading this tweet https//...,0


In [15]:
lista_teste = []
j = []
k = []
for f in g_limpo['Teste']:
    comR = 1
    comIr = 1
    x = f.split(' ')
    
    for palavra in x:
        
        if palavra in ir_prob:
            comIr *= ir_prob[palavra]
        else:
            comIr *= 1/(len(pala)+Irre)
        if palavra in r_prob:
            comR *= r_prob[palavra]
        else:
            comR *= 1/(len(pala)+Rel)

    Rp = comR*Pr
    Irp = comIr*Pi
    
    j.append(Rp)
    k.append(Irp)
    
    if Rp >= Irp:
        lista_teste.append(1)
    else:
        lista_teste.append(0)
min(k)
g["lista teste"] = lista_teste


In [16]:
uu = g.loc[(g['Relevante'] == 1 ) & (g['lista teste'] == 1 ), ['Relevante', 'lista teste']]
uz = g.loc[(g['Relevante'] == 0 ) & (g['lista teste'] == 1 ), ['Relevante', 'lista teste']]
zz = g.loc[(g['Relevante'] == 0 ) & (g['lista teste'] == 0 ), ['Relevante', 'lista teste']]
zu = g.loc[(g['Relevante'] == 1 ) & (g['lista teste'] == 0 ), ['Relevante', 'lista teste']]

posV = (len(uu)/len(g['Relevante']))*100
posF = (len(uz)/len(g['Relevante']))*100
negV = (len(zz)/len(g['Relevante']))*100
negF = (len(zu)/len(g['Relevante']))*100

print('Positivos verdadeiros = {:.3f}%'.format(posV))
print('Positivos falsos = {:.3f}%'.format(posF))
print('Negativos verdadeiros = {:.3f}%'.format(negV))
print('Negativos falsos = {:.3f}%'.format(negF))


Positivos verdadeiros = 44.000%
Positivos falsos = 20.000%
Negativos verdadeiros = 33.000%
Negativos falsos = 3.000%


P(Frase|relevante) = P(palavra|relevante) * P(outra_paravra|relevante)


# Mudança de índice - Bia fez

A mudança de iíndice foi realizada para que a primeira coluna do DataFrame fossem os tweets

In [17]:
tw_idx = tw.set_index('Treinamento')
tw_idx

,Relevante,Positivo/Negativo
Treinamento,,
lindt chocolate is so overrated,1.0,0
"@elgliko hear that pitter patter in the back ground every so often?\nthat’s me.\nin your costco.\nwhere i am living off a rounded diet of slim jim’s &amp; lindt chocolate, reading romance novels by the 6-pack, and having my pick of the giant pile of dog beds for some shuteye.\nyou’ll only find me",0.0,NaN
"rt @statutory_boy: the guy was addicted to drugs, his life is changed after he met this woman, god bless this woman ❤👊🙏 https://t.co/dcnehu…",0.0,NaN
"@andreyasasylum @yourmomsuckstho hey, can you please pass the last of that lindt dark chocolate?",1.0,1
great job by @noticgroup gms. come and see them and make sure you ask for a lindt ball. \n#signaturesolutions \n#wearecbr https://t.co/ynsmdfcxaj,1.0,1
rt @blacktwittasa_: lmao guys i'm screaming 🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣 https://t.co/sx7w2cfmvx,0.0,NaN
fuck i want candy or chocolate\n\ngoing ✈️ to spend all my money at the lindt store,1.0,1
@lindt_zwane thank you,1.0,NaN
@lindt_zwane misinterpret of of love sex money 💵 reliability patriarchy just ruined the whole meaning of true love,0.0,NaN


#### Separando Relevante e Irrelevante

In [18]:
rel = tw_idx[tw_idx.Relevante == 1]
rel

,Relevante,Positivo/Negativo
Treinamento,,
lindt chocolate is so overrated,1.0,0
"@andreyasasylum @yourmomsuckstho hey, can you please pass the last of that lindt dark chocolate?",1.0,1
great job by @noticgroup gms. come and see them and make sure you ask for a lindt ball. \n#signaturesolutions \n#wearecbr https://t.co/ynsmdfcxaj,1.0,1
fuck i want candy or chocolate\n\ngoing ✈️ to spend all my money at the lindt store,1.0,1
@lindt_zwane thank you,1.0,NaN
reebok emailed me to thank me for my collab posts with their shoes nothing else matters today\n\nexcept that lindt sent me ten truffle bags who wants to turn up w me,1.0,1
"😥 news - coffee, irish cream &amp; hazelnut gianduja lindt lindor's discontinued - get what's left while you can! https://t.co/ncphz2ypb7 https://t.co/hfslcab317",1.0,1
"@earwolf @mattgourley @tvsandydaly a new august lindt! whooohoooo! finally, and end to my sufferings!",1.0,1
@anewnamehere @verelaurent it's a chocolate museum by lindt. i haven't been there yet but one day i will! lol,1.0,1


In [19]:
irrel = tw_idx[tw_idx.Relevante == 0]
irrel

,Relevante,Positivo/Negativo
Treinamento,,
"@elgliko hear that pitter patter in the back ground every so often?\nthat’s me.\nin your costco.\nwhere i am living off a rounded diet of slim jim’s &amp; lindt chocolate, reading romance novels by the 6-pack, and having my pick of the giant pile of dog beds for some shuteye.\nyou’ll only find me",0.0,NaN
"rt @statutory_boy: the guy was addicted to drugs, his life is changed after he met this woman, god bless this woman ❤👊🙏 https://t.co/dcnehu…",0.0,NaN
rt @blacktwittasa_: lmao guys i'm screaming 🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣 https://t.co/sx7w2cfmvx,0.0,NaN
@lindt_zwane misinterpret of of love sex money 💵 reliability patriarchy just ruined the whole meaning of true love,0.0,NaN
how much out of 10 would you rate the 70% #darkchocolate version of #lindtexcellence blocks? https://t.co/aeppg6mlu4,0.0,NaN
"rt @curiouspeterg: how are unicorns fake but giraffes are real? like whats more believable, a horse with a horn or a leopard-moose-camel wi…",0.0,NaN
rt @crumbschester: 🍰 we are heading back to the @_makersmarket in chester this month. we’ve been working hard on new tasty treats lately &amp;…,0.0,NaN
co. i find them so sexy and intriguing. https://t.co/n5adazxwoa,0.0,NaN
rt @_bonga: guys. please put on your seatbelts when driving. i’ve just been informed that an old friend passed away yesterday morning after…,0.0,NaN


In [20]:
lista = tw.Treinamento.tolist()
print (lista)

['lindt chocolate is so overrated', '@elgliko hear that pitter patter in the back ground every so often?\nthat’s me.\nin your costco.\nwhere i am living off a rounded diet of slim jim’s &amp; lindt chocolate, reading romance novels by the 6-pack, and having my pick of the giant pile of dog beds for some shuteye.\nyou’ll only find me', 'rt @statutory_boy: the guy was addicted to drugs, his life is changed after he met this woman, god bless this woman ❤👊🙏 https://t.co/dcnehu…', '@andreyasasylum @yourmomsuckstho hey, can you please pass the last of that lindt dark chocolate?', 'great job by @noticgroup gms. come and see them and make sure you ask for a lindt ball.  \n#signaturesolutions  \n#wearecbr https://t.co/ynsmdfcxaj', "rt @blacktwittasa_: lmao guys i'm screaming 🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣 https://t.co/sx7w2cfmvx", 'fuck i want candy or chocolate\n\ngoing ✈️ to spend all my money at the lindt store', '@lindt_zwane thank you', '@lindt_zwane misinterpret of of love sex money 💵 reliability patriarchy

#### Criando as listas

In [21]:
lista = tw.groupby('Relevante').apply(lambda grupo: grupo.Treinamento.tolist()).tolist()
lista

[['@elgliko hear that pitter patter in the back ground every so often?\nthat’s me.\nin your costco.\nwhere i am living off a rounded diet of slim jim’s &amp; lindt chocolate, reading romance novels by the 6-pack, and having my pick of the giant pile of dog beds for some shuteye.\nyou’ll only find me',
  'rt @statutory_boy: the guy was addicted to drugs, his life is changed after he met this woman, god bless this woman ❤👊🙏 https://t.co/dcnehu…',
  "rt @blacktwittasa_: lmao guys i'm screaming 🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣 https://t.co/sx7w2cfmvx",
  '@lindt_zwane misinterpret of of love sex money 💵 reliability patriarchy just ruined the whole meaning of true love',
  'how much out of 10 would you rate the 70% #darkchocolate version of #lindtexcellence blocks? https://t.co/aeppg6mlu4',
  'rt @curiouspeterg: how are unicorns fake but giraffes are real? like whats more believable, a horse with a horn or a leopard-moose-camel wi…',
  'rt @crumbschester: 🍰 we are heading back to the @_makersmarket in chester t

In [22]:
lista_r = []
lista_r.append(lista[1])
lista_r

[['lindt chocolate is so overrated',
  '@andreyasasylum @yourmomsuckstho hey, can you please pass the last of that lindt dark chocolate?',
  'great job by @noticgroup gms. come and see them and make sure you ask for a lindt ball.  \n#signaturesolutions  \n#wearecbr https://t.co/ynsmdfcxaj',
  'fuck i want candy or chocolate\n\ngoing ✈️ to spend all my money at the lindt store',
  '@lindt_zwane thank you',
  'reebok emailed me to thank me for my collab posts with their shoes nothing else matters today\n\nexcept that lindt sent me ten truffle bags who wants to turn up w me',
  "😥 news - coffee, irish cream &amp; hazelnut gianduja lindt lindor's discontinued - get what's left while you can! https://t.co/ncphz2ypb7 https://t.co/hfslcab317",
  '@earwolf @mattgourley @tvsandydaly a new august lindt!  whooohoooo! finally, and end to my sufferings!',
  "@anewnamehere @verelaurent it's a chocolate museum by lindt. i haven't been there yet but one day i will! lol",
  'rt @gemsatwork: is this rea

In [23]:
lista_ir = []
lista_ir.append(lista[0])
lista_ir

[['@elgliko hear that pitter patter in the back ground every so often?\nthat’s me.\nin your costco.\nwhere i am living off a rounded diet of slim jim’s &amp; lindt chocolate, reading romance novels by the 6-pack, and having my pick of the giant pile of dog beds for some shuteye.\nyou’ll only find me',
  'rt @statutory_boy: the guy was addicted to drugs, his life is changed after he met this woman, god bless this woman ❤👊🙏 https://t.co/dcnehu…',
  "rt @blacktwittasa_: lmao guys i'm screaming 🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣 https://t.co/sx7w2cfmvx",
  '@lindt_zwane misinterpret of of love sex money 💵 reliability patriarchy just ruined the whole meaning of true love',
  'how much out of 10 would you rate the 70% #darkchocolate version of #lindtexcellence blocks? https://t.co/aeppg6mlu4',
  'rt @curiouspeterg: how are unicorns fake but giraffes are real? like whats more believable, a horse with a horn or a leopard-moose-camel wi…',
  'rt @crumbschester: 🍰 we are heading back to the @_makersmarket in chester t

In [24]:
lista_rel = []
for e in lista_r[0]:
    strings = e.split(' ')
    lista_rel.append(strings)
    
lista_rel

[['lindt', 'chocolate', 'is', 'so', 'overrated'],
 ['@andreyasasylum',
  '@yourmomsuckstho',
  'hey,',
  'can',
  'you',
  'please',
  'pass',
  'the',
  'last',
  'of',
  'that',
  'lindt',
  'dark',
  'chocolate?'],
 ['great',
  'job',
  'by',
  '@noticgroup',
  'gms.',
  'come',
  'and',
  'see',
  'them',
  'and',
  'make',
  'sure',
  'you',
  'ask',
  'for',
  'a',
  'lindt',
  'ball.',
  '',
  '\n#signaturesolutions',
  '',
  '\n#wearecbr',
  'https://t.co/ynsmdfcxaj'],
 ['fuck',
  'i',
  'want',
  'candy',
  'or',
  'chocolate\n\ngoing',
  '✈️',
  'to',
  'spend',
  'all',
  'my',
  'money',
  'at',
  'the',
  'lindt',
  'store'],
 ['@lindt_zwane', 'thank', 'you'],
 ['reebok',
  'emailed',
  'me',
  'to',
  'thank',
  'me',
  'for',
  'my',
  'collab',
  'posts',
  'with',
  'their',
  'shoes',
  'nothing',
  'else',
  'matters',
  'today\n\nexcept',
  'that',
  'lindt',
  'sent',
  'me',
  'ten',
  'truffle',
  'bags',
  'who',
  'wants',
  'to',
  'turn',
  'up',
  'w',
  'me

In [25]:
lista_irrel = []
for i in lista_ir[0]:
    string = i.split(' ')
    lista_irrel.append(string)
    
    
lista_irrel

[['@elgliko',
  'hear',
  'that',
  'pitter',
  'patter',
  'in',
  'the',
  'back',
  'ground',
  'every',
  'so',
  'often?\nthat’s',
  'me.\nin',
  'your',
  'costco.\nwhere',
  'i',
  'am',
  'living',
  'off',
  'a',
  'rounded',
  'diet',
  'of',
  'slim',
  'jim’s',
  '&amp;',
  'lindt',
  'chocolate,',
  'reading',
  'romance',
  'novels',
  'by',
  'the',
  '6-pack,',
  'and',
  'having',
  'my',
  'pick',
  'of',
  'the',
  'giant',
  'pile',
  'of',
  'dog',
  'beds',
  'for',
  'some',
  'shuteye.\nyou’ll',
  'only',
  'find',
  'me'],
 ['rt',
  '@statutory_boy:',
  'the',
  'guy',
  'was',
  'addicted',
  'to',
  'drugs,',
  'his',
  'life',
  'is',
  'changed',
  'after',
  'he',
  'met',
  'this',
  'woman,',
  'god',
  'bless',
  'this',
  'woman',
  '❤👊🙏',
  'https://t.co/dcnehu…'],
 ['rt',
  '@blacktwittasa_:',
  'lmao',
  'guys',
  "i'm",
  'screaming',
  '🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣🤣',
  'https://t.co/sx7w2cfmvx'],
 ['@lindt_zwane',
  'misinterpret',
  'of',
  'of',
  'love',
  'sex

#### Referências
* To list: https://pt.stackoverflow.com/questions/243921/criar-lista-com-conte%C3%BAdo-de-colunas

* Split: https://www.tutorialspoint.com/python/string_split.htm
* Tutorial